# Construyendo un sistema RAG multimodal con Docling

*Usando IBM Granite vision, embeddings de texto y modelos de IA generativa*


## Lab 3: Del Papel al Conocimiento para IA Transparente - El Viaje Completo
Bienvenido al laboratorio final de nuestra workshop de Docling! Has recorrido un largo camino:

- **Lab 1**: Aprendiste a convertir documentos preservando la estructura
- **Lab 2**: Dominaste estrategias inteligentes de chunking
- **Lab 3**: Ahora, construiremos un sistema RAG completo y listo para producción con una característica revolucionaria: el *visual grounding*


Este laboratorio representa la culminación de todo lo que has aprendido, mostrando cómo Docling permite no solo el procesamiento de documentos, sino sistemas de IA verdaderamente transparentes.

## ¿Por qué este laboratorio es importante?

Los sistemas RAG tradicionales tienen un problema de confianza. Cuando una IA proporciona información, los usuarios a menudo se preguntan:

- "¿De dónde proviene esta información?" 🔍
- "¿Puedo verificar que esto es preciso?" ✅
- "¿Está la IA alucinando o utilizando datos reales?" 🤔

**Visual Grounding** resuelve este problema mostrando a los usuarios exactamente de dónde se recuperó la información en los documentos originales. Esto no es solo una característica *agradable* de un sistema de IA, es esencial para casos de uso donde la precisión y la verificabilidad son cruciales:

- **Salud**: 🏥 Verificar fuentes de información médica
- **Legal**: ⚖️ Rastrear citas a ubicaciones exactas en documentos
- **Financiero**: 💰 Auditar ideas financieras generadas por IA
- **Investigación**: 🔬 Validar afirmaciones científicas
- **Empresarial**: 🏢 Construir sistemas de IA internos confiables


## ¿Qué hace especial a este lab?

Estamos construyendo un sistema RAG multimodal con *visual grounding* que:

1. **Procesa múltiples tipos de datos**: Texto, tablas e imágenes de tus documentos
2. **Muestra fuentes exactas**: Resalta la ubicación precisa de la información recuperada
3. **Comprende imágenes**: Utiliza modelos de visión por IA para comprender el contenido visual
4. **Mantiene la transparencia**: Cada respuesta puede ser verificada visualmente


---


## Entendiendo RAG Multimodal con Visual Grounding


### ¿Qué es RAG Multimodal?

[Retrieval Augmented Generation (RAG)](https://www.ibm.com/think/topics/retrieval-augmented-generation) es una técnica utilizada con LLMs para conectar el modelo con una base de conocimiento externa sin necesidad de realizar [fine-tuning](https://www.ibm.com/think/topics/rag-vs-fine-tuning).

Los sistemas RAG tradicionales están limitados a casos de uso basados en texto. Sin embargo, los documentos reales contienen:
- **Texto**: Párrafos, listas, encabezados
- **Tablas**: Datos estructurados, información financiera
- **Imágenes**: Gráficos, diagramas, fotos, ilustraciones

El RAG Multimodal puede utilizar [LLMs multimodales](https://www.ibm.com/think/topics/multimodal-ai) (MLLM) para procesar información de múltiples tipos de datos que se incluyen como parte de la base de conocimiento externa utilizada en RAG. Los datos multimodales pueden incluir texto, imágenes, audio, video u otras formas.


Puedes leer más sobre RAG Multimodal en este [artículo de IBM](https://www.ibm.com/think/topics/multimodal-rag).

### Visual Grounding: La Capa de Transparencia

El grounding agrega una capa crucial de transparencia a los sistemas RAG. Cuando el sistema recupera información para responder a una consulta, no solo devuelve texto, sino que también muestra exactamente de dónde proviene esa información en el documento original mediante:

- Dibujar cuadros delimitadores en las páginas del documento
- Resaltar regiones específicas
- Etiquetar tipos de contenido (TEXTO, TABLA, IMAGEN)
- Usar diferentes colores para múltiples fuentes

En este notebook, utilizarás los modelos IBM Granite, capaces de procesar diferentes modalidades, mejorados con las capacidades de grounding visual de Docling para crear un sistema de IA transparente y verificable.

---


## Objetivos

En este laboratorio, aprenderás a:

1. **Configurar Docling para el Grounding Visual**: Configurar el procesamiento de documentos para mantener referencias visuales
2. **Procesar Contenido Multimodal**: Manejar texto, tablas e imágenes con los metadatos adecuados
3. **Aprovechar los Modelos de Visión AI**: Utilizar los modelos de visión IBM Granite para entendimiento de imágenes
4. **Construir una Base de Datos Vectorial**: Almacenar embeddings con metadatos para visual grounding
5. **Implementar Atribución Visual**: Mostrar a los usuarios exactamente de dónde proviene la información
6. **Crear un Pipeline RAG Completo**: Combinar todos los componentes en un sistema listo para producción

### Tecnologías que usaremos

Componentes clave:

1. **[Docling](https://docling-project.github.io/docling/):** Un kit de herramientas de código abierto utilizado para analizar y convertir documentos.
2. **[LangChain](https://langchain.com)**: Para orquestar el pipeline RAG
3. **[IBM Granite Vision Models](https://www.ibm.com/granite/)**: Para entendimiento de contenido de imágenes
4. **Visual Grounding**: La capacidad única de Docling para atribución de fuentes


---

## Prerequisitos

Antes de comenzar, asegúrate de tener:
- Completados los Laboratorios 1 y 2 (o conocimiento equivalente de Docling)
- Python 3.10, 3.11 o 3.12 instalado
- Comprensión básica de embeddings y bases de datos vectoriales
- Familiaridad con los conceptos de los laboratorios anteriores

In [ ]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), "Use Python 3.10, 3.11, or 3.12 to run this notebook."

## Instalación de dependencias

In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    transformers \
    pillow \
    langchain_community \
    'langchain_huggingface[full]' \
    langchain_milvus 'pymilvus[milvus_lite]'\
    docling \
    replicate \
    matplotlib
! echo "::endgroup::"

## Importar las librerías necesarias

In [2]:
# To see detailed information about the document processing and visual grounding operations, we'll configure INFO log level.
# NOTE: It is okay to skip running this cell if you prefer less verbose output.
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
import json
import base64
import io
import itertools
import tempfile
from pathlib import Path
from tempfile import mkdtemp
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import PIL.ImageOps
from PIL import ImageDraw
from IPython.display import display

# Docling imports for document processing and visual grounding
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.document import DoclingDocument
from docling.chunking import DocMeta
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem, RefItem
from docling_core.types.doc.labels import DocItemLabel

# LangChain imports for RAG pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Replicate
from langchain_core.documents import Document
from langchain_core.vectorstores import VectorStore
from langchain_milvus import Milvus
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Model imports
from transformers import AutoTokenizer, AutoProcessor
from ibm_granite_community.notebook_utils import get_env_var, escape_f_string

ModuleNotFoundError: No module named 'langchain_huggingface'

---

### Selección de Modelos

### Los Tres Pilares de RAG Multimodal

Para un sistema completo de RAG multimodal con visual grounding, necesitamos tres tipos de modelos, cada uno cumpliendo un propósito crucial:

1. **Modelo de Embeddings**: Convierte texto en representaciones vectoriales
   - Permite la búsqueda semántica ("encontrar contenido similar en significado")
   - Debe manejar texto de fragmentos, tablas y descripciones de imágenes

2. **Modelo de Visión**: Entiende y describe contenido visual
   - Procesa imágenes encontradas en documentos
   - Genera descripciones textuales para su recuperación

3. **Modelo de Lenguaje**: Genera respuestas finales
   - Sintetiza la información recuperada
   - Produce respuestas coherentes y precisas

### Modelo de Embeddings

Usaremos un [modelo de embeddings Granite](https://huggingface.co/collections/ibm-granite/granite-embedding-models-6750b30c802c1926a35550bb) para generar vectores de embeddings de texto.


- Optimizado para texto en múltiples idiomas
- Compacto (107 millones de parámetros) para un procesamiento rápido
- Excelente comprensión semántica
- Ventana de contexto de 512 tokens

Si deseas usar un modelo diferente, puedes consultar [esta receta de modelos de embeddings de la comunidad de IBM Granite](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [ ]:
embeddings_model_path = "ibm-granite/granite-embedding-107m-multilingual"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)
print(f"Embeddings model loaded: {embeddings_model_path}")

### Cargar el Modelo de Visión Granite

El modelo de visión nos ayudará a comprender las imágenes dentro de los documentos. Esto es crucial para un RAG verdaderamente multimodal, ya que muchos documentos contienen información visual importante.

**¿Por qué usar un modelo alojado en la nube en lugar de uno local?**
- Procesamiento más rápido sin necesidad de GPU
- Rendimiento consistente
- Fácil de escalar

**Nota**: Para configurar Replicate, consulta [Introducción a Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb).

Para conectarte a un modelo en un proveedor diferente a Replicate, consulta la [receta de LLMs de Langchain de la comunidad de IBM Granite](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).


In [ ]:
vision_model_path = "ibm-granite/granite-vision-3.3-2b"
vision_model = Replicate(
    model=vision_model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": embeddings_tokenizer.max_len_single_sentence,
        "min_tokens": 100,
        "temperature": 0.01 # low temperature for reproduceability
    },
)
vision_processor = AutoProcessor.from_pretrained(vision_model_path)
print(f"Vision model loaded: {vision_model_path}")

### Cargar el Modelo de Lenguaje Granite

Finalmente, nuestro modelo de lenguaje generará respuestas basadas en el contexto recuperado.

**¿Por qué usar este modelo?**:
- 8B parámetros: Buen equilibrio entre calidad y velocidad
- Ajustado a instrucciones: Sigue nuestros prompts con precisión
- Familia Granite: Código abierto y con licencia Apache 2.0

In [ ]:
model_path = "ibm-granite/granite-3.3-8b-instruct"
model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": 1000,
        "min_tokens": 100,
        "temperature": 0.01 # low temperature for reproduceability

    },
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
print(f"Language model loaded: {model_path}")


---


## Procesamiento de documentos con soporte para visual grounding

El visual grounding requiere una configuración especial durante la conversión de documentos. A diferencia del procesamiento estándar, necesitamos:

1. **Generar imágenes de página de alta calidad**: Para resaltar elementos de la página visualmente.
2. **Preservar la información de coordenadas**: Para saber dónde se encuentra el contenido.
3. **Mantener la estructura del documento**: Para una atribución de fuentes precisa.
4. **Almacenar los documentos adecuadamente**: Para su posterior recuperación y visualización.

Vamos a configurar Docling con estos requisitos:


In [ ]:
# Configure the document converter to support visual grounding
pdf_pipeline_options = PdfPipelineOptions(
    do_ocr=False,  # Set to True if your PDFs contain scanned images
    generate_picture_images=True,  # Extract images from documents
    generate_page_images=True,  # CRITICAL: Generate page images for visual grounding
)

format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pdf_pipeline_options),
}

converter = DocumentConverter(format_options=format_options)
print("Document converter configured with visual grounding support")

### Creamos un almacén local de documentos para el visual grounding

Almacenaremos documentos que mantendrán la estructura completa del documento necesaria para el visual grounding. Esto es esencial para resaltar las coordenadas de origen más adelante:

In [ ]:
# Create document store for visual grounding
doc_store = {}
doc_store_root = Path(mkdtemp()) # Temporary directory for document store
print(f"Document store created at: {doc_store_root}")

### Convertir documentos con seguimiento visual

Ahora procesaremos documentos mientras preservamos toda la información necesaria para el visual grounding:


In [ ]:
sources = [
    "https://arxiv.org/pdf/2501.17887" # Docling paper
    # "https://arxiv.org/pdf/2206.01062", # DocLayNet paper
    # "https://arxiv.org/pdf/2311.18481", # DocQA
    # Añade más documentos según sea necesario
]

conversions = {}

print("Iniciando la conversión de documentos con visual grounding...")
for source in sources:
    # Por cada fuente, convertimos el documento preservando las imágenes y guardándolas en nuestro almacén local de documentos
    print(f"\n Procesando: {source}")

    # Convert document
    result = converter.convert(source=source)
    docling_document = result.document
    conversions[source] = docling_document

    # Save document to store for visual grounding
    # The binary hash ensures unique identification
    file_path = doc_store_root / f"{docling_document.origin.binary_hash}.json"
    docling_document.save_as_json(file_path)
    doc_store[docling_document.origin.binary_hash] = file_path

    print("Documento convertido y guardado en el almacén local.")
    print(f"  - Document ID: {docling_document.origin.binary_hash}")
    print(f"  - Paginas: {len(docling_document.pages)}")
    print(f"  - Tablas: {len(docling_document.tables)}")
    print(f"  - Imágenes: {len(docling_document.pictures)}")

## Procesado del Contenido del Documento con Metadatos de Atribución


### La Importancia de los Metadatos para el Visual Grounding

Para que el visual grounding funcione, cada fragmento de contenido debe mantener metadatos sobre su ubicación de origen. Esto incluye:
- **Números de página**: Qué página(s) contienen este contenido
- **Cajas delimitadoras**: Coordenadas exactas en la página
- **Referencias de documentos**: Enlaces de regreso al documento fuente
- **Tipo de contenido**: Si es texto, tabla o imagen

Este metadato es lo que nos permite resaltar regiones de interés específicas en las páginas del documento más adelante.

## Procesamos los Chunks de Texto con metadatos de ubicación


Ahora procesamos cualquier tabla en los documentos. Convertimos los datos de la tabla al formato markdown para pasarlos al modelo de lenguaje. Se crea una lista de documentos LangChain a partir de las representaciones en markdown de la tabla.

In [ ]:
from docling.chunking import DocMeta

doc_id = 0
texts: list[Document] = []

print("\nProcessing text chunks with visual grounding metadata...")
for source, docling_document in conversions.items():
    chunker = HybridChunker(tokenizer=embeddings_tokenizer)

    for chunk in chunker.chunk(docling_document):
        items = chunk.meta.doc_items

        # Skip single-item chunks that are tables (we'll process them separately)
        if len(items) == 1 and isinstance(items[0], TableItem):
            continue

        refs = " ".join(map(lambda item: item.get_ref().cref, items))
        text = chunk.text

        # Create document with enhanced metadata for visual grounding
        document = Document( # langchain_core.documents.Document
            page_content=text,
            metadata={
                "doc_id": (doc_id:=doc_id+1),
                "source": source,
                "ref": refs,  # References for tracking specific document items
                "dl_meta": chunk.meta.model_dump(),  # CRITICAL: Store chunk metadata for visual grounding
                "origin_hash": docling_document.origin.binary_hash  # Link to stored document
            },
        )
        texts.append(document)

print(f"Created {len(texts)} text chunks with visual grounding metadata")

### Procesamos las tablas con información espacial

Las tablas requieren un manejo especial para preservar su estructura y ubicación:

In [ ]:
doc_id = len(texts)
tables: list[Document] = []

print("\nProcessing tables...")
for source, docling_document in conversions.items():
    for table in docling_document.tables:
        if table.label in [DocItemLabel.TABLE]:
            ref = table.get_ref().cref
            text = table.export_to_markdown(docling_document)

            # Extract provenance information for visual grounding
            prov_data = []
            if hasattr(table, 'prov') and table.prov:
                for prov in table.prov:
                    # Get the page to access its height for coordinate conversion
                    if prov.page_no < len(docling_document.pages):
                        page = docling_document.pages[prov.page_no]
                        # Convert to top-left origin and normalize
                        bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                        bbox_norm = bbox.normalized(page.size)

                        prov_data.append({
                           "page_no": prov.page_no,
                           "bbox": {
                              "l": bbox_norm.l,  # Use normalized coordinates
                              "t": bbox_norm.t,
                              "r": bbox_norm.r,
                              "b": bbox_norm.b
                        }
                    })

            document = Document(
                page_content=text,
                metadata={
                    "doc_id": (doc_id:=doc_id+1),
                    "source": source,
                    "ref": ref,
                    "origin_hash": docling_document.origin.binary_hash,
                    "item_type": "table",  # Mark as table
                    "prov_data": prov_data  # Store provenance as simple data
                },
            )
            tables.append(document)

print(f"Created {len(tables)} table documents")

In [ ]:
print(tables[0].page_content)

### Procesamos las imágenes con entendimiento visual

Para un verdadero RAG multimodal, necesitamos entender el contenido de las imágenes. Usaremos el modelo de visión Granite para generar descripciones.

**NOTA**: El procesamiento de imágenes puede llevar tiempo dependiendo de la cantidad de imágenes y del servicio del modelo de visión. Cada imagen será analizada individualmente.

In [ ]:
def encode_image(image: PIL.Image.Image, format: str = "png") -> str:
    """Encode image to base64 for vision model processing"""
    image = PIL.ImageOps.exif_transpose(image) or image
    image = image.convert("RGB")

    buffer = io.BytesIO()
    image.save(buffer, format)
    encoding = base64.b64encode(buffer.getvalue()).decode("utf-8")
    uri = f"data:image/{format};base64,{encoding}"
    return uri

# Configuración del prompt de visión - siéntete libre de experimentar con esto!
image_prompt = "Give a detailed description of what is depicted in the image"
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": image_prompt},
        ],
    },
]
vision_prompt = vision_processor.apply_chat_template(
    conversation=conversation,
    add_generation_prompt=True,
)

pictures: list[Document] = []
doc_id = len(texts) + len(tables)

for source, docling_document in conversions.items():
    num_pictures = len(docling_document.pictures)
    for i, picture in enumerate(docling_document.pictures):
        ref = picture.get_ref().cref
        print(f"  Processing image: {ref} ({i+1}/{num_pictures})")

        image = picture.get_image(docling_document)
        if image:
            # Generate image description using vision model
            text = vision_model.invoke(vision_prompt, image=encode_image(image))

            # Extract provenance information for visual grounding
            prov_data = []
            if hasattr(picture, 'prov') and picture.prov:
                for prov in picture.prov:
                    # Get the page to access its height for coordinate conversion
                    if prov.page_no < len(docling_document.pages):
                        page = docling_document.pages[prov.page_no]
                        # Convert to top-left origin and normalize
                        bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                        bbox_norm = bbox.normalized(page.size)

                        prov_data.append({
                            "page_no": prov.page_no,
                            "bbox": {
                                "l": bbox_norm.l,
                                "t": bbox_norm.t,
                                "r": bbox_norm.r,
                                "b": bbox_norm.b
                            }
                        })

            document = Document(
                page_content=text,
                metadata={
                    "doc_id": (doc_id:=doc_id+1),
                    "source": source,
                    "ref": ref,
                    "origin_hash": docling_document.origin.binary_hash,
                    "item_type": "picture",  # Mark as picture for special handling
                    "prov_data": prov_data  # Store normalized provenance data
                },
            )
            pictures.append(document)

print(f"Created {len(pictures)} image descriptions")

In [ ]:
def encode_image(image: PIL.Image.Image, format: str = "png") -> str:
    """Encode image to base64 for vision model processing"""
    image = PIL.ImageOps.exif_transpose(image) or image
    image = image.convert("RGB")

    buffer = io.BytesIO()
    image.save(buffer, format)
    encoding = base64.b64encode(buffer.getvalue()).decode("utf-8")
    uri = f"data:image/{format};base64,{encoding}"
    return uri

### Mostramos una muestra de los documentos procesados

Examinemos lo que hemos creado para entender la naturaleza multimodal de nuestro sistema:

In [ ]:
import textwrap

print("\nSample processed documents:")
print("=" * 80)

# Show sample text chunks
print("\nTEXT CHUNK EXAMPLES:")
print("-" * 80)
for i, text_doc in enumerate(texts[:3]):  # Show first 3 text chunks
    print(f"\nText Chunk {i+1}:")
    print(f"  Document ID: {text_doc.metadata['doc_id']}")
    print(f"  Source: {text_doc.metadata['source'].split('/')[-1]}")  # Just filename
    print(f"  Reference: {text_doc.metadata['ref']}")
    print(f"  Has visual grounding: {'dl_meta' in text_doc.metadata}")
    print("  Content preview:")
    print(f"    {text_doc.page_content[:250]}...")
    if i < 2:  # Add separator between examples except after the last one
        print("-" * 40)

# Show sample tables
print("\n\nTABLE EXAMPLES:")
print("-" * 80)
if tables:
    for i, table_doc in enumerate(tables[:3]):  # Show first 3 tables
        print(f"\nTable {i+1}:")
        print(f"  Document ID: {table_doc.metadata['doc_id']}")
        print(f"  Reference: {table_doc.metadata['ref']}")
        print("  Content preview (Markdown format):")
        # Show first few lines of the table
        table_lines = table_doc.page_content.split('\n')[:8]
        for line in table_lines:
            print(f"    {line}")
else:
    print("  No tables found in the document.")

# Show sample images with descriptions
print("\n\nIMAGE EXAMPLES WITH AI-GENERATED DESCRIPTIONS:")
print("-" * 80)
if pictures:
    for i, pic_doc in enumerate(pictures[:3]):  # Show first 3 images
        print(f"\nImage {i+1}:")
        print(f"  Document ID: {pic_doc.metadata['doc_id']}")
        print(f"  Reference: {pic_doc.metadata['ref']}")
        print("  AI-Generated Description:")
        # Wrap the description for better readability
        wrapped_text = textwrap.fill(pic_doc.page_content, width=70, initial_indent="    ", subsequent_indent="    ")
        print(wrapped_text)

        # Display the actual image
        source = pic_doc.metadata['source']
        ref = pic_doc.metadata['ref']
        docling_document = conversions[source]
        picture = RefItem(cref=ref).resolve(docling_document)
        image = picture.get_image(docling_document)
        if image:
            print("\n  Original Image:")
            # Resize image for display if too large
            display_image = image.copy()
            max_width = 600
            if display_image.width > max_width:
                ratio = max_width / display_image.width
                new_height = int(display_image.height * ratio)
                display_image = display_image.resize((max_width, new_height), PIL.Image.Resampling.LANCZOS)
            display(display_image)

        if i < min(2, len(pictures)-1):
            print("-" * 40)
else:
    print("  No images found in the document.")

### Comprendiendo la Atribución Visual de Fuentes

El grounding visual es lo que diferencia a este sistema. Las funciones que definiremos en las siguientes celdas nos permiten:
1. **Localizar**: Encontrar la fuente exacta de cualquier información recuperada
2. **Resaltar**: Dibujar indicadores visuales en las páginas del documento
3. **Diferenciar**: Usar estilos distintos para texto, tablas e imágenes
4. **Verificar**: Permitir a los usuarios confirmar las respuestas de la IA contra los documentos fuente


In [ ]:
# Esta función visualiza de dónde proviene un fragmento de texto en el documento original.
def visualize_chunk_grounding(chunk, doc_store, highlight_color="blue"):
    """
    Visualize where a text chunk comes from in the original document.

    This function:
    1. Loads the original document from the store
    2. Finds the pages containing the chunk content
    3. Draws bounding boxes around the source regions
    4. Displays the highlighted pages

    Args:
        chunk: LangChain Document with visual grounding metadata
        doc_store: Dictionary mapping document hashes to file paths
        highlight_color: Color for highlighting (blue, green, red, etc.)

    Returns:
        Dictionary of page images with highlights
    """
    # Get the origin hash
    origin_hash = chunk.metadata.get("origin_hash")
    if not origin_hash:
        print("No origin hash found in metadata")
        return None

    # Load the full document from store
    dl_doc = DoclingDocument.load_from_json(doc_store.get(origin_hash))

    print(f"Visualizing source location for chunk {chunk.metadata.get('doc_id', 'Unknown')}")

    # Handle different types of content
    page_images = {}
    item_type = chunk.metadata.get("item_type", "text")

    if item_type in ["picture", "table"] and "prov_data" in chunk.metadata:
        # Handle tables and pictures with simple provenance data
        prov_data = chunk.metadata["prov_data"]

        if not prov_data:
            print(f"No provenance data available for this {item_type}")
            return None

        for prov in prov_data:
            page_no = prov["page_no"]

            # Get page image
            if page_no < len(dl_doc.pages):
                page = dl_doc.pages[page_no]
                if hasattr(page, 'image') and page.image:
                    if page_no not in page_images:
                        img = page.image.pil_image.copy()
                        page_images[page_no] = {
                            'image': img,
                            'page': page,
                            'draw': ImageDraw.Draw(img)
                        }

                    # Draw bounding box
                    draw = page_images[page_no]['draw']
                    bbox = prov["bbox"]

                    # Draw bounding box
                    draw = page_images[page_no]['draw']
                    bbox = prov["bbox"]

                    # The coordinates are already normalized and in top-left origin
                    # Just scale to image dimensions
                    img_width = page_images[page_no]['image'].width
                    img_height = page_images[page_no]['image'].height

                    l = int(bbox["l"] * img_width)
                    r = int(bbox["r"] * img_width)
                    t = int(bbox["t"] * img_height)
                    b = int(bbox["b"] * img_height)

                    # Ensure coordinates are valid (min/max) just in case
                    l, r = min(l, r), max(l, r)
                    t, b = min(t, b), max(t, b)

                    # Clamp to image bounds
                    l = max(0, min(l, img_width - 1))
                    r = max(0, min(r, img_width - 1))
                    t = max(0, min(t, img_height - 1))
                    b = max(0, min(b, img_height - 1))

                    # Draw highlight with different styles for different types
                    if item_type == "picture":
                        draw.rectangle([l, t, r, b], outline=highlight_color, width=4)
                        draw.text((l, t-20), "IMAGE", fill=highlight_color)
                    elif item_type == "table":
                        draw.rectangle([l, t, r, b], outline=highlight_color, width=3)
                        draw.text((l, t-20), "TABLE", fill=highlight_color)

    elif "dl_meta" in chunk.metadata:
        # Handle text chunks with DocMeta
        try:
            meta = DocMeta.model_validate(chunk.metadata["dl_meta"])

            # Process each item in the chunk to find source locations
            for doc_item in meta.doc_items:
                if hasattr(doc_item, 'prov') and doc_item.prov:
                    for prov in doc_item.prov:
                        page_no = prov.page_no

                        # Get or create page image
                        if page_no not in page_images:
                            if page_no < len(dl_doc.pages):
                                page = dl_doc.pages[page_no]
                                if hasattr(page, 'image') and page.image:
                                    img = page.image.pil_image.copy()
                                    page_images[page_no] = {
                                        'image': img,
                                        'page': page,
                                        'draw': ImageDraw.Draw(img)
                                    }

                        # Draw bounding box on the page
                        if page_no in page_images:
                            page_data = page_images[page_no]
                            page = page_data['page']
                            draw = page_data['draw']

                            # Convert coordinates to image space
                            bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                            bbox = bbox.normalized(page.size)

                            # Scale to actual image dimensions
                            l = int(bbox.l * page_data['image'].width)
                            r = int(bbox.r * page_data['image'].width)
                            t = int(bbox.t * page_data['image'].height)
                            b = int(bbox.b * page_data['image'].height)

                            # Draw highlight rectangle
                            draw.rectangle([l, t, r, b], outline=highlight_color, width=2)
        except Exception as e:
            print(f"Error processing text chunk metadata: {e}")
            return None
    else:
        print("No visual grounding metadata available for this chunk")
        return None

    # Display highlighted pages
    for page_no, page_data in sorted(page_images.items()):
        plt.figure(figsize=(12, 16))
        plt.imshow(page_data['image'])
        plt.axis('off')

        # Add title indicating content type
        if item_type == "picture":
            title = "Image Location"
        elif item_type == "table":
            title = "Table Location"
        else:
            title = "Text Location"
        plt.title(f'{title} - Page {page_no + 1}', fontsize=16)
        plt.tight_layout()
        plt.show()

    return page_images


## Popular la base de datos vectorial con embeddings y metadatos

### Comprendiendo las Bases de Datos Vectoriales en RAG Multimodal

Las bases de datos vectoriales son el motor de búsqueda de nuestro sistema RAG. Estas:
- Almacenan representaciones numéricas (embeddings) de nuestro contenido
- Permiten búsquedas por similitud semántica
- Mantienen todos los metadatos necesarios para el grounding visual
- Soportan una recuperación rápida a escala

Para contenido multimodal, esto significa:
- Los fragmentos de texto se incrustan directamente
- El markdown de las tablas se incrusta para búsquedas estructurales
- Las descripciones de imágenes generadas por IA se incrustan para búsquedas visuales


### Configuración de la Base de Datos Vectorial

Usaremos Milvus, una base de datos vectorial de alto rendimiento. Milvus es una base de datos de código abierto diseñada para manejar grandes volúmenes de datos vectoriales, lo que la hace ideal para aplicaciones de IA y aprendizaje automático. Para más opciones de bases de datos vectoriales, consulta [esta receta de Almacenamiento Vectorial con Langchain](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [ ]:
# Create a temporary database file
db_file = tempfile.NamedTemporaryFile(prefix="vectorstore_", suffix=".db", delete=False).name
print(f"Vector database will be saved to: {db_file}")

# Initialize Milvus vector store
vector_db: VectorStore = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    enable_dynamic_field=True,  # Allows flexible metadata storage
    index_params={"index_type": "AUTOINDEX"},  # Automatic index optimization
)


### Añadimos Documentos a la Base de Datos Vectorial

Ahora añadiremos todos nuestros documentos procesados (fragmentos de texto, tablas y descripciones de imágenes) a la base de datos vectorial:

In [ ]:
print("\nAñadiendo documentos a la base de datos vectorial...")
documents = list(itertools.chain(texts, tables, pictures))
ids = vector_db.add_documents(documents)
print(f"Añadidos {len(ids)} documentos a la base de datos vectorial.")
print(f"  - Fragmentos de texto: {len(texts)}")
print(f"  - Tablas: {len(tables)}")
print(f"  - Descripciones de imágenes: {len(pictures)}")

## Test de Recuperación con Atribución Visual

### Probando la Recuperación con Atribución Visual

Antes de construir todo el pipeline de RAG, probemos que nuestra recuperación y atribución visual funcionen correctamente. Esto ayuda a verificar:
- Se encuentra contenido basado en similitud semántica
- Se preserva la metadata de atribución visual
- Se manejan correctamente diferentes tipos de contenido

### Basic Retrieval Test


In [ ]:
# Test query
test_query = "How much was spent on food distribution relative to the amount of food distributed?"

print(f"\nTesting retrieval for query: '{test_query}'")
print("=" * 80)

# Retrieve relevant documents
retrieved_docs = vector_db.as_retriever().invoke(test_query)

# Display retrieved documents
for i, doc in enumerate(retrieved_docs):
    print(f"\nRetrieved Document {i+1}:")

    # Determine content type
    item_type = doc.metadata.get('item_type', 'text')
    if item_type == 'picture':
        content_type = "AI-Generated Image Description"
    elif item_type == 'table':
        content_type = "Table (Markdown)"
    else:
        content_type = "Text Chunk"

    print(f"Type: {content_type}")
    print(f"Content preview: {doc.page_content[:200]}...")
    print(f"Source: {doc.metadata['source'].split('/')[-1]}")

## Construcción del Pipeline RAG Completo


Ahora implementaremos el sistema RAG multimodal completo que:
1. Recupera contenido multimodal relevante
2. Muestra exactamente de dónde proviene cada pieza
3. Genera respuestas precisas y fundamentadas

### Diferenciando Tipos de Contenido

Nuestro sistema maneja tres tipos de contenido de manera distinta:

1. **Fragmentos de Texto**: El resaltado estándar muestra pasajes de texto
2. **Tablas**: Bordes gruesos con etiquetas "TABLE" marcan datos estructurados
3. **Imágenes**: Bordes distintivos con etiquetas "IMAGE" muestran ubicaciones de imágenes

Esta diferenciación visual ayuda a los usuarios a comprender rápidamente qué tipo de contenido contribuyó a la respuesta de la IA.

### Probando el Sistema RAG Multimodal con Visual Grounding

In [ ]:
# Bringing It All Together
def rag_with_visual_grounding(question, vector_db, doc_store, model, tokenizer, top_k=5):
    """
    Perform RAG with visual grounding of results.

    This function:
    1. Retrieves relevant chunks from the vector database
    2. Visualizes where each chunk comes from in the original document
    3. Generates a response using the retrieved context

    Args:
        question: User's query
        vector_db: Vector database with embedded documents
        doc_store: Document store for visual grounding
        model: Language model for response generation
        tokenizer: Tokenizer for the language model
        top_k: Number of chunks to retrieve

    Returns:
        Tuple of (outputs, relevant_chunks)
    """
    print(f"\nPregunta: {question}")
    print("=" * 80)

    # Step 1: Retrieve relevant chunks
    print(f"\nRecuperando los {top_k} fragmentos relevantes...")
    retriever = vector_db.as_retriever(search_kwargs={"k": top_k})
    relevant_chunks = retriever.invoke(question)

    print(f"Se encontraron {len(relevant_chunks)} fragmentos relevantes")

    # Step 2: Visualize each chunk's source location
    print("\nVisualizando la ubicación de los fragmentos recuperados...")

    for i, chunk in enumerate(relevant_chunks):
        print(f"\n--- Resultado {i+1} ---")

        # Determine content type
        item_type = chunk.metadata.get('item_type', 'text')
        if item_type == 'picture':
            content_type = "Descripción de Imagen Generada por IA"
            color = 'red'
        elif item_type == 'table':
            content_type = "Tabla (Markdown)"
            color = 'green'
        else:
            content_type = "Fragmento de Texto"
            color = 'blue'

        print(f"Content type: {content_type}")
        print(f"Text preview: {chunk.page_content[:200]}...")
        print(f"Source: {chunk.metadata.get('source', 'Unknown').split('/')[-1]}")

        # Show visual grounding if available
        if "dl_meta" in chunk.metadata or "prov_data" in chunk.metadata:
            visualize_chunk_grounding(
                chunk,
                doc_store,
                highlight_color=color
            )
        else:
            print("  (No visual grounding available for this chunk)")

    # Step 3: Create RAG pipeline for response generation
    print("\nGenerando respuesta con LLM...")

    # Create Granite prompt template
    prompt = tokenizer.apply_chat_template(
        conversation=[{
            "role": "system",
            "content": "Answer the question based on the provided context in a concise manner. Always answer in the same language as the question."
        },
        {
            "role": "user",
            "content": "{input}",
        }],
        documents=[{
            "doc_id": "0",
            "text": "{context}",
        }],
        add_generation_prompt=True,
        tokenize=False,
    )

    prompt_template = PromptTemplate.from_template(
        template=escape_f_string(prompt, "input", "context")
    )

    # Document prompt template
    document_prompt_template = PromptTemplate.from_template(template="""\
<|end_of_text|>
<|start_of_role|>document {{"document_id": "{doc_id}"}}<|end_of_role|>
{page_content}""")

    # Create chains
    combine_docs_chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt_template,
        document_prompt=document_prompt_template,
        document_separator="",
    )

    rag_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=combine_docs_chain,
    )

    # Generate response
    outputs = rag_chain.invoke({"input": question})

    print("\nRespuesta generada:")
    print("=" * 80)
    print(outputs['answer'])
    print("=" * 80)

    return outputs, relevant_chunks

## Demostración Final


Vamos a ejecutar una consulta y ver el sistema completo en acción:

In [ ]:
main_query = "Como funciona la pipeline de conversión de PDFs de Docling?"
outputs, chunks = rag_with_visual_grounding(
    main_query,
    vector_db,
    doc_store,
    model,
    tokenizer,
    top_k=3
)

# Resumen y Próximos Pasos

### Lo que Has Logrado

¡Felicidades! Has construido con éxito un sistema avanzado de RAG multimodal. Esto es lo que has aprendido:

1. **Implementación de un RAG con *Visual Grounding***
   - Configuraste Docling para preservar referencias visuales
   - Mantuviste metadatos de coordenadas a lo largo del pipeline de procesamiento
   - Creaste atribución visual para todos los tipos de contenido

2. **Procesamiento de Documentos Multimodal**
   - Manejo fluido de texto, tablas e imágenes
   - Uso de chunking inteligente para optimizar la recuperación
   - Uso de modelos de visión IA para comprensión de imágenes

3. **Arquitectura Transparente de RAG**
   - Como establecer confianza en tu sistema RAG mediante verificación visual
   - Habilitación de atribución de fuentes para cada respuesta
   - Creación de un resaltado diferenciado por cada tipo de contenido

4. **Integración Lista para Producción**
   - Combinación efectiva de múltiples modelos de IA
   - Creación de un pipeline escalable de procesamiento de documentos

### ¿Por qué el Grounding Visual lo Cambia Todo?

Los sistemas RAG tradicionales son "cajas negras": los usuarios deben confiar ciegamente en la IA. Tu sistema:

- **Muestra las fuentes**: Cada afirmación puede ser verificada visualmente
- **Construye confianza**: Los usuarios ven exactamente de dónde proviene la información
- **Habilita auditorías**: Perfecto para industrias reguladas
- **Reduce alucinaciones**: La verificación visual detecta errores

### El Poder de la Comprensión Multimodal

Al procesar texto, tablas e imágenes, tu sistema:

- Captura información completa del documento
- Habilita consultas y respuestas más ricas
- Maneja la complejidad de documentos del mundo real
- Proporciona respuestas completas

---

## Proximos Pasos:


1. **Experimenta con otros documentos**
   - Prueba con documentos en español, francés o alemán
   - Prueba documentos con diagramas técnicos y gráficos
   - Procesa informes con contenido mixto

2. **Personaliza la IA para ti**
   - Utiliza los modelos de embeddings, visión y lenguaje que se adapten a tu flujo de trabajo común.
   - Ajusta los prompts para mejorar la calidad de las descripciones de imágenes y respuestas del modelo de lenguaje.
   
   ```python
   # Ejemplo: Prompts de imagen específicos a un dominio
   medical_prompt = "Describe esta imagen médica, señalando cualquier anomalía o característica clave"
   financial_prompt = "Analiza este gráfico financiero, identificando tendencias y puntos de datos clave"
   ```

3. **Optimiza el Rendimiento**
   - Procesa documentos en batch: [Docling Batch Conversion](https://docling-project.github.io/docling/examples/batch_convert/)
   - Usa aceleración por GPU con modelos de visión locales.

---

## Recursos Adicionales

### Documentación Oficial
- **[Documentación de Docling](https://github.com/docling-project/docling)**: Últimas características y actualizaciones
- **[Modelos IBM Granite](https://www.ibm.com/granite/)**: Tarjetas de modelo y capacidades
- **[Documentación de LangChain](https://python.langchain.com/)**: Patrones y mejores prácticas de RAG
- **[Documentación de Milvus](https://milvus.io/docs)**: Optimización de bases de datos vectoriales

### Recursos Comunitarios
- Únete a la [comunidad de Docling en GitHub](https://github.com/docling-project/docling/discussions)
- Comparte tus implementaciones
- Contribuye con mejoras Docling! ❤️
  
### Temas Relacionados para Explorar
- Análisis de Diseño de Documentos
- Embeddings Multimodales
- Respuesta a Preguntas Visuales
- Sistemas de IA Explicables

---

## Conclusión

Has completado un increíble viaje desde la conversión básica de documentos hasta la construcción de un sistema de IA sofisticado y transparente. La combinación de la comprensión de documentos de Docling, las capacidades de IA de Granite y el grounding visual crea una aplicación poderosa.

Tu sistema RAG multimodal representa la vanguardia de la IA para documentos. Ya sea que estés construyendo para el sector de la salud, legal, financiero o cualquier otro dominio, ahora tienes las herramientas para crear sistemas de IA que no solo son poderosos, sino también confiables y transparentes.